# Backend Benchmark: PySide6 vs GLFW

This notebook benchmarks import and load times for different GUI backends with `run_gui`.

In [1]:
!uv pip install jupyterlab-vim

Using Python 3.12.9 environment at: C:\Users\RBO\repos\mbo_utilities\.venv
Resolved 93 packages in 216ms
Installed 1 package in 32ms
 + jupyterlab-vim==4.1.4


In [3]:
import time
import subprocess
import sys
from pathlib import Path

In [4]:
# Test data path - update this to your test data
TEST_DATA_PATH = r"E:\tests\lbm\mbo_utilities\big_raw"

# Number of runs for averaging
N_RUNS = 3

## 1. Import Time Benchmarks

Measure how long it takes to import the necessary modules for each backend.

In [5]:
def benchmark_import(backend: str, n_runs: int = 3) -> dict:
    """Benchmark import times for a specific backend."""
    
    script = f'''
import time
import os
os.environ["WGPU_GUI_BACKEND"] = "{backend}"

start = time.perf_counter()
import wgpu
wgpu_time = time.perf_counter() - start

start = time.perf_counter()
import fastplotlib as fpl
fpl_time = time.perf_counter() - start

start = time.perf_counter()
import mbo_utilities as mbo
mbo_time = time.perf_counter() - start

start = time.perf_counter()
from mbo_utilities.graphics import run_gui
run_gui_time = time.perf_counter() - start

print(f"wgpu:{{wgpu_time:.4f}}")
print(f"fpl:{{fpl_time:.4f}}")
print(f"mbo:{{mbo_time:.4f}}")
print(f"run_gui:{{run_gui_time:.4f}}")
'''
    
    times = {"wgpu": [], "fpl": [], "mbo": [], "run_gui": []}
    
    for i in range(n_runs):
        result = subprocess.run(
            [sys.executable, "-c", script],
            capture_output=True,
            text=True,
            env={**dict(__import__("os").environ), "WGPU_GUI_BACKEND": backend}
        )
        
        if result.returncode != 0:
            print(f"Error on run {i+1}: {result.stderr}")
            continue
            
        for line in result.stdout.strip().split("\n"):
            key, val = line.split(":")
            times[key].append(float(val))
    
    # Calculate averages
    return {
        "backend": backend,
        "wgpu_avg": sum(times["wgpu"]) / len(times["wgpu"]) if times["wgpu"] else 0,
        "fpl_avg": sum(times["fpl"]) / len(times["fpl"]) if times["fpl"] else 0,
        "mbo_avg": sum(times["mbo"]) / len(times["mbo"]) if times["mbo"] else 0,
        "run_gui_avg": sum(times["run_gui"]) / len(times["run_gui"]) if times["run_gui"] else 0,
        "total_avg": sum(sum(v) for v in times.values()) / n_runs if any(times.values()) else 0,
    }

In [6]:
print("Benchmarking GLFW imports...")
glfw_import = benchmark_import("glfw", N_RUNS)
print(f"GLFW Import Times (avg of {N_RUNS} runs):")
print(f"  wgpu:    {glfw_import['wgpu_avg']:.4f}s")
print(f"  fpl:     {glfw_import['fpl_avg']:.4f}s")
print(f"  mbo:     {glfw_import['mbo_avg']:.4f}s")
print(f"  run_gui: {glfw_import['run_gui_avg']:.4f}s")
print(f"  TOTAL:   {glfw_import['total_avg']:.4f}s")

Benchmarking GLFW imports...
GLFW Import Times (avg of 3 runs):
  wgpu:    0.0975s
  fpl:     1.9428s
  mbo:     1.6063s
  run_gui: 0.0185s
  TOTAL:   3.6651s


In [7]:
print("Benchmarking PySide6 imports...")
pyside6_import = benchmark_import("pyside6", N_RUNS)
print(f"PySide6 Import Times (avg of {N_RUNS} runs):")
print(f"  wgpu:    {pyside6_import['wgpu_avg']:.4f}s")
print(f"  fpl:     {pyside6_import['fpl_avg']:.4f}s")
print(f"  mbo:     {pyside6_import['mbo_avg']:.4f}s")
print(f"  run_gui: {pyside6_import['run_gui_avg']:.4f}s")
print(f"  TOTAL:   {pyside6_import['total_avg']:.4f}s")

Benchmarking PySide6 imports...
PySide6 Import Times (avg of 3 runs):
  wgpu:    0.0808s
  fpl:     1.9347s
  mbo:     1.5933s
  run_gui: 0.0188s
  TOTAL:   3.6277s


## 2. Data Load + Widget Creation Benchmarks

Measure how long it takes to load data and create the ImageWidget.

In [8]:
def benchmark_widget_creation(backend: str, data_path: str, n_runs: int = 3) -> dict:
    """Benchmark widget creation times for a specific backend."""
    
    script = f'''
import time
import os
os.environ["WGPU_GUI_BACKEND"] = "{backend}"

# Import phase
import_start = time.perf_counter()
import mbo_utilities as mbo
from mbo_utilities.graphics.run_gui import _create_image_widget
import_time = time.perf_counter() - import_start

# Data load phase
load_start = time.perf_counter()
data = mbo.imread(r"{data_path}")
load_time = time.perf_counter() - load_start

# Widget creation phase (without showing)
widget_start = time.perf_counter()
import fastplotlib as fpl
import numpy as np
from mbo_utilities.graphics._processors import MboImageProcessor

ndim = data.ndim
if ndim == 4:
    slider_dim_names = ("t", "z")
    window_funcs = (np.mean, None)
    window_sizes = (1, None)
elif ndim == 3:
    slider_dim_names = ("t",)
    window_funcs = (np.mean,)
    window_sizes = (1,)
else:
    slider_dim_names = None
    window_funcs = None
    window_sizes = None

iw = fpl.ImageWidget(
    data=data,
    processors=MboImageProcessor,
    slider_dim_names=slider_dim_names,
    window_funcs=window_funcs,
    window_sizes=window_sizes,
    histogram_widget=True,
    figure_kwargs={{"size": (800, 800)}},
    graphic_kwargs={{"vmin": -100, "vmax": 4000}},
)
widget_time = time.perf_counter() - widget_start

# Show phase
show_start = time.perf_counter()
iw.show()
show_time = time.perf_counter() - show_start

# First frame render
render_start = time.perf_counter()
iw.figure.canvas.draw()
render_time = time.perf_counter() - render_start

# Cleanup
try:
    iw.close()
except:
    pass

print(f"import:{{import_time:.4f}}")
print(f"load:{{load_time:.4f}}")
print(f"widget:{{widget_time:.4f}}")
print(f"show:{{show_time:.4f}}")
print(f"render:{{render_time:.4f}}")
print(f"shape:{{data.shape}}")
'''
    
    times = {"import": [], "load": [], "widget": [], "show": [], "render": []}
    shape = None
    
    for i in range(n_runs):
        print(f"  Run {i+1}/{n_runs}...", end=" ")
        result = subprocess.run(
            [sys.executable, "-c", script],
            capture_output=True,
            text=True,
            env={**dict(__import__("os").environ), "WGPU_GUI_BACKEND": backend}
        )
        
        if result.returncode != 0:
            print(f"Error: {result.stderr[:200]}")
            continue
        
        print("done")
        for line in result.stdout.strip().split("\n"):
            key, val = line.split(":")
            if key == "shape":
                shape = val
            else:
                times[key].append(float(val))
    
    # Calculate averages
    return {
        "backend": backend,
        "shape": shape,
        "import_avg": sum(times["import"]) / len(times["import"]) if times["import"] else 0,
        "load_avg": sum(times["load"]) / len(times["load"]) if times["load"] else 0,
        "widget_avg": sum(times["widget"]) / len(times["widget"]) if times["widget"] else 0,
        "show_avg": sum(times["show"]) / len(times["show"]) if times["show"] else 0,
        "render_avg": sum(times["render"]) / len(times["render"]) if times["render"] else 0,
    }

In [9]:
print(f"Benchmarking GLFW widget creation with data: {TEST_DATA_PATH}")
glfw_widget = benchmark_widget_creation("glfw", TEST_DATA_PATH, N_RUNS)
print(f"\nGLFW Widget Creation Times (avg of {N_RUNS} runs):")
print(f"  Data shape: {glfw_widget['shape']}")
print(f"  Import:     {glfw_widget['import_avg']:.4f}s")
print(f"  Load data:  {glfw_widget['load_avg']:.4f}s")
print(f"  Widget:     {glfw_widget['widget_avg']:.4f}s")
print(f"  Show:       {glfw_widget['show_avg']:.4f}s")
print(f"  Render:     {glfw_widget['render_avg']:.4f}s")
glfw_total = sum([glfw_widget['import_avg'], glfw_widget['load_avg'], 
                  glfw_widget['widget_avg'], glfw_widget['show_avg'], glfw_widget['render_avg']])
print(f"  TOTAL:      {glfw_total:.4f}s")

Benchmarking GLFW widget creation with data: E:\tests\lbm\mbo_utilities\big_raw
  Run 1/3... Error: 
Counting frames: 100%|##########| 49/49 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "<string>", line 56, in <module>
AttributeEr
  Run 2/3... Error: 
Counting frames: 100%|##########| 49/49 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "<string>", line 56, in <module>
AttributeEr
  Run 3/3... Error: 
Counting frames: 100%|##########| 49/49 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "<string>", line 56, in <module>
AttributeEr

GLFW Widget Creation Times (avg of 3 runs):
  Data shape: None
  Import:     0.0000s
  Load data:  0.0000s
  Widget:     0.0000s
  Show:       0.0000s
  Render:     0.0000s
  TOTAL:      0.0000s


In [10]:
print(f"Benchmarking PySide6 widget creation with data: {TEST_DATA_PATH}")
pyside6_widget = benchmark_widget_creation("pyside6", TEST_DATA_PATH, N_RUNS)
print(f"\nPySide6 Widget Creation Times (avg of {N_RUNS} runs):")
print(f"  Data shape: {pyside6_widget['shape']}")
print(f"  Import:     {pyside6_widget['import_avg']:.4f}s")
print(f"  Load data:  {pyside6_widget['load_avg']:.4f}s")
print(f"  Widget:     {pyside6_widget['widget_avg']:.4f}s")
print(f"  Show:       {pyside6_widget['show_avg']:.4f}s")
print(f"  Render:     {pyside6_widget['render_avg']:.4f}s")
pyside6_total = sum([pyside6_widget['import_avg'], pyside6_widget['load_avg'], 
                     pyside6_widget['widget_avg'], pyside6_widget['show_avg'], pyside6_widget['render_avg']])
print(f"  TOTAL:      {pyside6_total:.4f}s")

Benchmarking PySide6 widget creation with data: E:\tests\lbm\mbo_utilities\big_raw
  Run 1/3... Error: 
Counting frames: 100%|##########| 49/49 [00:00<?, ?it/s]
Ignoring invalid WGPU_GUI_BACKEND 'pyside6', must be one of ['glfw', 'qt', 'jupyter', 
  Run 2/3... Error: 
Counting frames: 100%|##########| 49/49 [00:00<?, ?it/s]
Ignoring invalid WGPU_GUI_BACKEND 'pyside6', must be one of ['glfw', 'qt', 'jupyter', 
  Run 3/3... Error: 
Counting frames: 100%|##########| 49/49 [00:00<?, ?it/s]
Ignoring invalid WGPU_GUI_BACKEND 'pyside6', must be one of ['glfw', 'qt', 'jupyter', 

PySide6 Widget Creation Times (avg of 3 runs):
  Data shape: None
  Import:     0.0000s
  Load data:  0.0000s
  Widget:     0.0000s
  Show:       0.0000s
  Render:     0.0000s
  TOTAL:      0.0000s


## 3. Summary Comparison

In [ ]:
print("=" * 60)
print("BACKEND COMPARISON SUMMARY")
print("=" * 60)
print(f"\nData: {TEST_DATA_PATH}")
print(f"Shape: {glfw_widget.get('shape', 'N/A')}")
print(f"Runs per benchmark: {N_RUNS}")
print()

print("-" * 60)
print(f"{'Metric':<20} {'GLFW':>15} {'PySide6':>15} {'Diff':>10}")
print("-" * 60)

# Import times
diff = pyside6_import['total_avg'] - glfw_import['total_avg']
sign = "+" if diff > 0 else ""
print(f"{'Import (total)':<20} {glfw_import['total_avg']:>14.4f}s {pyside6_import['total_avg']:>14.4f}s {sign}{diff:>9.4f}s")

# Widget creation breakdown
metrics = [
    ("Data Load", "load_avg"),
    ("Widget Create", "widget_avg"),
    ("Show", "show_avg"),
    ("First Render", "render_avg"),
]

for label, key in metrics:
    g = glfw_widget.get(key, 0)
    p = pyside6_widget.get(key, 0)
    diff = p - g
    sign = "+" if diff > 0 else ""
    print(f"{label:<20} {g:>14.4f}s {p:>14.4f}s {sign}{diff:>9.4f}s")

print("-" * 60)

# Total comparison
glfw_total = glfw_import['total_avg'] + sum(glfw_widget.get(k, 0) for _, k in metrics)
pyside6_total = pyside6_import['total_avg'] + sum(pyside6_widget.get(k, 0) for _, k in metrics)
diff = pyside6_total - glfw_total
sign = "+" if diff > 0 else ""
pct = (diff / glfw_total) * 100 if glfw_total > 0 else 0

print(f"{'TOTAL':<20} {glfw_total:>14.4f}s {pyside6_total:>14.4f}s {sign}{diff:>9.4f}s")
print()

if diff > 0:
    print(f"GLFW is {abs(pct):.1f}% faster than PySide6")
else:
    print(f"PySide6 is {abs(pct):.1f}% faster than GLFW")

## 4. Optional: Frame Rate Benchmark

Measure rendering performance by timing slider updates.

In [ ]:
def benchmark_frame_rate(backend: str, data_path: str, n_frames: int = 100) -> dict:
    """Benchmark frame update times."""
    
    script = f'''
import time
import os
os.environ["WGPU_GUI_BACKEND"] = "{backend}"

import mbo_utilities as mbo
import fastplotlib as fpl
import numpy as np
from mbo_utilities.graphics._processors import MboImageProcessor

data = mbo.imread(r"{data_path}")

ndim = data.ndim
if ndim == 4:
    slider_dim_names = ("t", "z")
    window_funcs = (np.mean, None)
    window_sizes = (1, None)
elif ndim == 3:
    slider_dim_names = ("t",)
    window_funcs = (np.mean,)
    window_sizes = (1,)
else:
    slider_dim_names = None
    window_funcs = None
    window_sizes = None

iw = fpl.ImageWidget(
    data=data,
    processors=MboImageProcessor,
    slider_dim_names=slider_dim_names,
    window_funcs=window_funcs,
    window_sizes=window_sizes,
    histogram_widget=False,
    figure_kwargs={{"size": (800, 800)}},
    graphic_kwargs={{"vmin": -100, "vmax": 4000}},
)
iw.show()

# Warm up
for i in range(5):
    iw.indices["t"] = i
    iw.figure.canvas.draw()

# Benchmark
n_frames = min({n_frames}, data.shape[0] - 1)
start = time.perf_counter()
for i in range(n_frames):
    iw.indices["t"] = i
    iw.figure.canvas.draw()
elapsed = time.perf_counter() - start

try:
    iw.close()
except:
    pass

fps = n_frames / elapsed
print(f"frames:{n_frames}")
print(f"elapsed:{elapsed:.4f}")
print(f"fps:{fps:.2f}")
'''
    
    print(f"  Running {n_frames} frame updates...", end=" ")
    result = subprocess.run(
        [sys.executable, "-c", script],
        capture_output=True,
        text=True,
        env={**dict(__import__("os").environ), "WGPU_GUI_BACKEND": backend}
    )
    
    if result.returncode != 0:
        print(f"Error: {result.stderr[:200]}")
        return {"backend": backend, "fps": 0, "elapsed": 0, "frames": 0}
    
    print("done")
    data = {}
    for line in result.stdout.strip().split("\n"):
        key, val = line.split(":")
        data[key] = float(val)
    
    return {
        "backend": backend,
        "fps": data.get("fps", 0),
        "elapsed": data.get("elapsed", 0),
        "frames": int(data.get("frames", 0)),
    }

In [ ]:
# Uncomment to run frame rate benchmarks (takes longer)

# print("Benchmarking GLFW frame rate...")
# glfw_fps = benchmark_frame_rate("glfw", TEST_DATA_PATH, n_frames=100)
# print(f"GLFW: {glfw_fps['fps']:.2f} FPS ({glfw_fps['frames']} frames in {glfw_fps['elapsed']:.2f}s)")

# print("\nBenchmarking PySide6 frame rate...")
# pyside6_fps = benchmark_frame_rate("pyside6", TEST_DATA_PATH, n_frames=100)
# print(f"PySide6: {pyside6_fps['fps']:.2f} FPS ({pyside6_fps['frames']} frames in {pyside6_fps['elapsed']:.2f}s)")